# DBSCAN Clustering Analysis
Студент: Казакевич Анна Юрьевна
Группа: 5030102/10201

### Теоретическая справка
Алгоритм DBSCAN (Density-Based Spatial Clustering of Applications with Noise) используется для кластеризации данных на основе плотности.
Он выделяет группы (кластеры) точек, которые расположены близко друг к другу, а точки, находящиеся далеко от любых кластеров, считаются шумом.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import seaborn as sns

### 1. Загрузка данных
Загрузка данных из файла data.csv для дальнейшего анализа и обработки.

In [ ]:
data_path = "data.csv"
df = pd.read_csv(data_path)
df.head()

### 2. Первичный анализ данных
Посмотрим на первые строки данных, чтобы понять их структуру.
Также проверим типы данных, пропуски и статистическую информацию.

In [ ]:
# Первый взгляд на данные
df.head()

# Информация о данных
df.info()

# Статистика
df.describe()

# Проверка на пропуски
df.isnull().sum()

### 3. Визуализация данных
Построим гистограммы для изучения распределения признаков и матрицу корреляции для анализа связей между ними.

In [ ]:
# Гистограммы
df.hist(bins=20, figsize=(12, 10))
plt.suptitle("Гистограммы данных", fontsize=16)
plt.show()

# Матрица корреляции
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Матрица корреляции признаков")
plt.show()

### 4. Предварительная обработка данных
Удалим категориальные признаки, заполним пропуски средними значениями и нормализуем данные.

In [ ]:
# Удаление категориальных признаков
data = df.drop(columns=["Channel", "Region"])

# Заполнение пропусков
data.fillna(data.mean(), inplace=True)

# Нормализация данных
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

### 5. Определение оптимального значения eps
Оптимальное значение eps определяем с помощью графика расстояний до 5-го ближайшего соседа.

In [ ]:
# Поиск ближайших соседей
neighbors = NearestNeighbors(n_neighbors=5)
neighbors_fit = neighbors.fit(data_scaled)
distances, indices = neighbors_fit.kneighbors(data_scaled)

# Сортировка расстояний
distances = np.sort(distances[:, 4], axis=0)

# Построение графика
plt.figure(figsize=(8, 5))
plt.plot(distances)
plt.title("Определение оптимального значения eps")
plt.xlabel("Точки данных")
plt.ylabel("Расстояние до 5-го ближайшего соседа")
plt.grid()
plt.show()

### 6. Настройка и запуск DBSCAN
Запустим DBSCAN с различными значениями eps и min_samples.
Для визуализации кластеров используем метод PCA для проекции данных в 2D-пространство.

In [ ]:
# Определение значений для экспериментов
eps_v = [1.5, 2, 2.5]
min_samples_v = [3, 10, 15]

# Визуализация результатов кластеризации
for min_samples in min_samples_v:
    for eps in eps_v:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        clusters = dbscan.fit_predict(data_scaled)

        # Добавление кластеров в исходный DataFrame
        df['Cluster'] = clusters

        # Преобразование данных в пространство из 2 компонент
        pca_2d = PCA(n_components=2)
        data_2d = pca_2d.fit_transform(data_scaled)
        df['PCA1'] = data_2d[:, 0]
        df['PCA2'] = data_2d[:, 1]

        # Визуализация кластеров
        plt.figure(figsize=(10, 7))
        sns.scatterplot(
            x='PCA1', y='PCA2', hue='Cluster', data=df, 
            palette='viridis', legend="full"
        )
        plt.title(f"Кластеры DBSCAN при eps = {eps}, min_samples = {min_samples}")
        plt.show()